# Блокнот-шаблон для работы на Julia в Google Colab Notebook

Порядок действий:
- Пересохраните себе блокнот *Файл -> Сохранить копию на Диске*
- При необходимости подключите аппаратный ускоритель *Среда выполнения -> Сменить среду выполнения -> Аппаратный ускоритель*
- Отредактируйте параметры установки 
  - `JULIA_VERSION` - версия Julia
  - `JULIA_PACKAGES` - предустанавливаемые пакеты
  - `JULIA_PACKAGES_IF_GPU` - пакеты для работы на ГПУ
  - `JULIA_NUM_THREADS` - число потоков
- Выполните первую ячейку
- Перезагрузите страницу (*F5*)

In [ ]:
%%shell 
set -e 
 
#---------------------------------------------#
JULIA_VERSION="1.6.1"
JULIA_PACKAGES="BenchmarkTools PyPlot"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------#
 
if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then 
  
  # Установка Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"` 
  echo "Установка Julia $JULIA_VERSION..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz
  tar xf /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz
   
  # Установка ядра и переименование на "julia"
  echo "Установка ядра IJulia..."
  julia -e 'using Pkg; Pkg.add("IJulia"); using IJulia; 
  IJulia.installkernel("julia", env=Dict("JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  echo "Переименование ядра на 'julia'..."
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*` 
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia
    
  # Установка дополнительных пакетов
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do                        
    echo "Установка Julia-пакета $PKG..."
    julia -e 'using Pkg; Pkg.add("'"$PKG"'")'
  done 
       
  echo ''
  echo "Вуаля! Перезагрузите страницу прежде, чем приступить к работе."
fi